In [18]:
using Distributions
using Statistics

mutable struct Particle
    params :: Vector{Float64}
    dist :: Float64
end


function limit(params, priors)
    [max(minimum(d), min(maximum(d), p)) for (p, d) in zip(params, priors)]
end
        

function abc(priors, dist_func, pop_size, p_rem, sigma, n_iters; verbose = false, parallel = false, scale_noise = false)
    particles = Particle[]
    new_particles = [Particle(map(rand, priors), Inf) for i in 1:pop_size]
        
    iter = 0
    
    while true
        iter += 1
        verbose && println("$iter: simulating $(length(new_particles)) new particles...")
        
        if parallel
            Threads.@threads for p in new_particles
                p.dist = dist_func(p.params)
            end        
        else
            for p in new_particles
                p.dist = dist_func(p.params)
            end        
        end
            
        # add new particles to old ones
        particles = [particles ; new_particles]
        
        if iter >= n_iters
            return particles
        end

        rem = floor(Int, p_rem * length(particles))
    
        verbose && println("removing $rem of $(length(particles))")
        # remove worst particles
        sort!(particles, by=p->p.dist)
        particles = particles[1:(end-rem)]
        
        verbose && println("distance: ", particles[1].dist, " ", particles[end].dist)
        
        
        if scale_noise
            std_dev = [std([p.params[i] for p in particles]) for i in eachindex(priors)]
            n_sigma = sigma .* std_dev
        else
            n_sigma = sigma
        end
        
        noise = Normal.(0, n_sigma)
        
        s = particles[end].dist + particles[1].dist
        weights = cumsum([s - p.dist for p in particles])
        sel = Uniform(0, weights[end])

        empty!(new_particles)
        
        for i in 1:pop_size
            anc = particles[searchsortedfirst(weights, rand(sel))]
            params = anc.params .+ rand.(noise)
            params = limit(params, priors)
            push!(new_particles, Particle(params, Inf))
        end
    end
            
            
end

abc (generic function with 1 method)

In [2]:
prior = [Normal(0.5), Normal(0.5)]

function cost1(p)
    x,y = p
    50 * (x + randn() * 0.01 - y^2)^2 + (y - 1 + randn()*0.01)^2
end

#@time res = abc(prior, cost1, 100, 0.5, 0.05, 7, verbose=false);
# mean([p.params[1] for p in res[1:100]])

In [19]:
for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
    for n in [0.01, 0.05, 0.1]
        d = 0.0
        for i in 1:100
            res = abc(prior, cost1, 200, p, [n, n], 10, verbose=false)
            d += res[100].dist
        end
        print("\t$(d/100)")
    end
    println()
end     

	0.4598261711667547	0.4265687594284958	0.43391580692400017
	0.16759781515053368	0.17381587448392077	0.2496085290203974
	0.056995302149243976	0.09400811904204884	0.15991029219631078
	0.028239501029437505	0.05733614790459573	0.11502088316526721
	0.02675645174334148	0.03813044741878595	0.09096945596469369
	0.010822490737860964	0.026639285157002604	0.06419023266968808


In [16]:
for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
    for n in [0.01, 0.05, 0.1]
        d = 0.0
        for i in 1:100
            res = abc(prior, cost1, 200, p, [n, n], 10, verbose=false, scale_noise=true)
            d += res[100].dist
        end
        print("\t$(d/100)")
    end
    println()
end     

	0.4064794205453891	0.4052369774250329	0.42051580753168266
	0.19359271304960984	0.1565097545763017	0.16476830767958725
	0.07696924998127548	0.05676474486905425	0.08214105866205816
	0.03919851799640959	0.01781104766044468	0.03572449862876857
	0.04167741893109759	0.008369152749395557	0.014473769113515143
	0.03375561934320172	0.004576521327811295	0.005646908436767691


In [20]:
using KissABC

prior = Factored(Normal(0,5), Normal(0,5))

n = 1

function cost2((x,y))
    global n
    n += 1
    50 * (x + randn() * 0.01 - y^2)^2 + (y - 1 + randn()*0.01)^2
end

@time results = smc(prior, cost2, alpha=0.6, verbose = true);

(iteration, ϵ, dest, target) = (1, 33550.21019632241, 65.99999999999999, 60.00000000000001)
(iteration, ϵ, dest, target) = (2, 4462.37039793619, 42.0, 39.59999999999999)
(iteration, ϵ, dest, target) = (3, 1150.8145771291436, 59.99999999999998, 60.0)
(iteration, ϵ, dest, target) = (4, 297.7097081354878, 60.00000000000001, 60.0)
(iteration, ϵ, dest, target) = (5, 90.78720368864866, 60.000000000000014, 60.0)
(iteration, ϵ, dest, target) = (6, 40.7335607712651, 37.00000000000001, 36.00000000000001)
(iteration, ϵ, dest, target) = (7, 20.317504079402042, 59.000000000000014, 60.0)
(iteration, ϵ, dest, target) = (8, 8.804982344442564, 59.000000000000014, 60.0)
(iteration, ϵ, dest, target) = (9, 4.671366075094421, 61.000000000000036, 60.0)
(iteration, ϵ, dest, target) = (10, 1.929769239734854, 37.000000000000014, 36.60000000000002)
(iteration, ϵ, dest, target) = (11, 0.9804807217732695, 57.0, 60.0)
(iteration, ϵ, dest, target) = (12, 0.7082080650304935, 61.000000000000014, 60.0)
(iteration, ϵ, 

In [21]:
n

2981